# It is running for the long time and I became out of my patience. So why I stopped running the code in the middle and got no output. But perhaps the code is right.
# If this type of error is showing - " ModuleNotFoundError: No module named 'tensorflow' " - Then run this code before doing everything " !pip install tensorflow "

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD, Adam

# Load dataset
df = pd.read_csv("iris.csv")

# Encode labels
encoder = LabelEncoder()
df['species'] = encoder.fit_transform(df['species'])

# Features and target
X = df.drop('species', axis=1)
y = to_categorical(df['species'])

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

def build_model(activation='relu', loss='categorical_crossentropy', optimizer='adam'):
    model = Sequential()
    model.add(Dense(64, input_shape=(4,), activation=activation))
    for _ in range(4):  # 5 hidden layers total
        model.add(Dense(64, activation=activation))
    model.add(Dense(3, activation='softmax'))  # 3 classes

    model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
    return model

# Configurable parameters
activation = 'relu'
loss_fn = 'categorical_crossentropy'  # or 'mse'
optimizer = Adam(learning_rate=0.01)
epochs = 300

model = build_model(activation=activation, loss=loss_fn, optimizer=optimizer)
history = model.fit(X_train, y_train, epochs=epochs, validation_split=0.2, verbose=0)

# Evaluate
loss, acc = model.evaluate(X_test, y_test)
print(f"\nTest Accuracy: {acc:.4f} | Test Loss: {loss:.4f}")

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label="Train Loss")
plt.plot(history.history['val_loss'], label="Val Loss")
plt.title("Loss Curve")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label="Train Accuracy")
plt.plot(history.history['val_accuracy'], label="Val Accuracy")
plt.title("Accuracy Curve")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()

plt.tight_layout()
plt.show()

# Predict classes
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

# Classification report
print("\nClassification Report:\n")
print(classification_report(y_true_classes, y_pred_classes, target_names=encoder.classes_))

# Confusion Matrix Heatmap
conf_matrix = confusion_matrix(y_true_classes, y_pred_classes)
sns.heatmap(conf_matrix, annot=True, cmap="Blues", xticklabels=encoder.classes_, yticklabels=encoder.classes_)
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()

print("\nSample Weights from First Layer:")
weights = model.layers[0].get_weights()[0]
print(weights[:3])  # first 3 neurons’ weights
